<h1 align="center"> <span style="font-size: 20px;"> Proyecto Final Grupo Nº 7 </span> <br>
    <align="center"> <span style="font-size: 15px;">  Proyecto Yelp & Google Maps - Reseñas y Recomendaciones</span><br>         
<align="center"> <span style="font-size: 15px;">@utores:  Javier Castro, Luca Ramallo, Luis Ramírez, Lesmen García.</span> <br>
  <a href="https://github.com/lesmengp/Proyecto-Final-Grupo-07.git">GitHub: <span style="font-size: 20px;">Proyecto Yelp & Google Maps</span></a> 
</h1>

### Librerías a Utilizar:

In [ ]:
!pip install google.cloud

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import os
import io
import json
import gcsfs

## Ejemplos de Análisis de Sentimientos con NLTK y VADER, desde BigQuery

### Cargando desde BigQuery el Dataset: df_yelp_reviews

In [61]:
from google.cloud import bigquery
client = bigquery.Client()
sql = """
    SELECT * FROM `eternal-empire-399016.gmy_bq.df_yelp_reviews` LIMIT 1000000
"""
df_yelp_reviews = client.query(sql).to_dataframe()
df_yelp_reviews.head(5)

,review_id,user_id,business_id,stars,useful,cool,text,date
0,7ADnyQu_R0J5RXsplp4yRw,huj0zcSIIy7FkZAE3WYvwA,n4PdRDOWgBOhke7AqCW3FA,1,2,0,This place is absolutely horrible. Food took 1...,2018-01-01
1,9bwn1x6aa-d-s24RTZybaA,TBkyvXeENnwx9Mf_inWU-w,zuGpUPcfws1FeupSkt8WVw,1,0,0,If I could rate this place lower I would. I we...,2018-01-01
2,nTu-Rl3hP7L6ucsf_f252A,EZdGX7ok9PDhW6Xy0kM-gg,qVTTPVz4W91yJiQMrK-R1A,1,0,0,Interesting that a few months ago this was a t...,2018-01-01
3,Ibv8SQHib6lQ5-tP8NLQzw,9wp7Wbuxoqs1T-t5rOwo3g,317lIGWVcxcVWdVo6ZKHwg,1,4,1,Wow...New Years day and what a mess. Lines are...,2018-01-01
4,MpuNFagYZBRaA0_GP5cDtA,qsq5BYe1bTrQiqu5NmbnOQ,M7Y338g9JFs_0fNR1RA82w,1,0,0,I went on New Year's Day and I understand not ...,2018-01-01


In [1]:
from google.cloud import bigquery
client = bigquery.Client()
sql = """
    SELECT * FROM `eternal-empire-399016.gmy_bq.df_yelp_reviews`
"""
df_yelp_reviews = client.query(sql).to_dataframe()
df_yelp_reviews.head(5)

,review_id,user_id,business_id,stars,useful,cool,text,date
0,a4y-vnBF6wra6o5q_14kRQ,3UFFy3y9sYiVFJCJoaROgw,pcqHaOUL50eh7CYBqQ1XEA,5,7,1,Sauce on the Side originates from Missouri. St...,2019-08-04
1,CI-L5IQCxJjP5jZf0hRERQ,y_nRpTpmsteDzhdGKLnXiw,ssk6Q7dOtOBP-8ovssi1jw,5,7,5,We always enjoyed our dine in experiences here...,2020-11-02
2,kdiUyAbEToh8gTFTs8CRSg,OIDpKC2p61qVO_spPqw1qA,368cyArdo0P0aDUOhhE4PA,5,13,10,I'm not even exaggerating a little bit when I ...,2019-11-28
3,GHi343ONGwc4bU0hho5ecw,61yERKmEa_0mkZYy_3PNAg,6cnHwjHLcAtIuMN0C6Kpbw,5,7,3,This was such a fun event! I love that it was ...,2019-06-13
4,k8vCNrNvFk6LI0_bxmqJuA,-t16WTKzFVzbC3sJ2EW3Uw,b36EeutJRvowZMr9TpQm0w,5,10,5,Oat Milk? In Reno? At a drive thru? Color me a...,2020-07-24


In [3]:
df_yelp_reviews.shape

(2713751, 8)

In [4]:
df_yelp_reviews.columns

Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'cool',
       'text', 'date'],
      dtype='object')

In [63]:
df_yelp_reviews.sample(5)

,review_id,user_id,business_id,stars,useful,cool,text,date
346728,LBTUHdqcVLOcpmBJucMRTg,EGS2AMsxME0HWAWDZhDFiA,_I9SL8ycMEJBJiV4bGYPnA,1,0,0,This hotel is filthy. There were cigarette bur...,2019-06-22
403792,IyMZMVawFoAc9N-uVQSNHw,1FGmmjOsyyzS1_AsOIv6KA,QvB0NRTGDplwnfPZGq8chw,2,4,1,The food was mediocre after having paid 15.00 ...,2018-05-19
972402,v9MjveLpqqXET0z1zc7iOw,mXNMLwOoIuN559fmD8d2ig,rjDbYdgCIDZcdJs25xIAIQ,1,0,0,Was considering going here until online not on...,2018-07-17
315357,nxeoILAxuxUA8bN3_LouZA,6SrDnjcBVhRrin85tbwDFA,ClC5I3aOLhK4WcVPOJMYIA,5,0,0,Traditional steakhouse. Love the dark wood amb...,2021-03-13
180883,4d-6Qm7CNyT74PsKYssZ4g,7b2SEu7IoNHknIkwUV2hWg,dEHHpeN5XDXLhMBGNzrl8g,4,0,0,What can I say that hasn't already been said. ...,2018-05-20


In [5]:
import nltk
nltk.download('vader_lexicon')   # VADER es una Biblioteca que ya contirnr algunas reglas para el Analisis de Sentimientos. Asigna puntuaciones
from nltk.sentiment import SentimentIntensityAnalyzer  # Nos da una probabilidad si el mensaje es negativo, neutro o positivo. 

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [6]:
sia = SentimentIntensityAnalyzer() # Se instancia el modelo

In [7]:
def AnalizarSentimiento(texto):
    sia = SentimentIntensityAnalyzer() # Se instancia el modelo
    sentimiento = sia.polarity_scores(texto)["compound"]
    
    if sentimiento > 0.1:
        return "Sentimiento Positivo"
    else:
        if sentimiento < 0:
            return "Sentimiento Negativo"
        else:
            return "Sentimiento Neutro"

In [32]:
df = df_yelp_reviews.sample(5).copy()
len(df)

5

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, 568 to 738
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   review_id    5 non-null      object
 1   user_id      5 non-null      object
 2   business_id  5 non-null      object
 3   stars        5 non-null      Int64 
 4   useful       5 non-null      Int64 
 5   cool         5 non-null      Int64 
 6   text         5 non-null      object
 7   date         5 non-null      object
dtypes: Int64(3), object(5)
memory usage: 375.0+ bytes


### Probando la Funcion de Sentimientos NLTK con VADER, con N registros aleatorios...!!!

In [8]:
### Probando solo con 100 Registros
N = 100000   # Número de Registro.
df = df_yelp_reviews.sample(N).copy()   # Extrae Aleatoriamente N registros del Dataset original
df['Sentimiento'] = df['text'].apply(AnalizarSentimiento)
df[['text', 'Sentimiento']].head(10)

,text,Sentimiento
2446030,Was here to pick up a dozen of regular cupcake...,Sentimiento Positivo
2640775,"I like my countertops, but everything else was...",Sentimiento Negativo
1418842,Not only is the staff fabulous during your app...,Sentimiento Positivo
984020,Food was good but the service was slow. They d...,Sentimiento Positivo
950902,I loved this place while we lived in Nash. I t...,Sentimiento Positivo
760844,Amber Indian Buffet re-opens in Castleton! We'...,Sentimiento Positivo
730444,Can't speak to the customer service is the kio...,Sentimiento Positivo
312292,Went in to discuss upgrading a set of rings. M...,Sentimiento Positivo
2639050,5 stars for MICKY!!! Oh and also the same for ...,Sentimiento Positivo
386573,If you are thinking of trying this place for T...,Sentimiento Negativo


### Tiempo de ejecuciòn del c
### Inicio: 12:48
### Fin: 12:59
### Tardo 11 minutos